<a href="https://colab.research.google.com/github/misiek447/merito-2025/blob/main/football%20data%20project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Import niezbędnych bibliotek
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# (opcjonalnie, do wstępnej wizualizacji)
import seaborn as sns

# ustawienia dla czytelnych wykresów
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

In [3]:
# 2.1. Wczytanie pliku CSV do DataFrame
df = pd.read_csv('/content/results_cleaned.csv')   # lub ścieżka, pod którą wrzuciłeś plik

# 2.2. Rzut oka na strukturę danych
print(df.shape)        # ile wierszy i kolumn?
df.info()              # typy kolumn i braki danych
display(df.head(5))    # pierwsze 5 wierszy

# 2.3. Podstawowe statystyki opisowe
display(df.describe(include='all'))

(27065, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27065 entries, 0 to 27064
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        27065 non-null  object
 1   home_team   27065 non-null  object
 2   away_team   27065 non-null  object
 3   home_score  27065 non-null  int64 
 4   away_score  27065 non-null  int64 
 5   tournament  27065 non-null  object
 6   city        27065 non-null  object
 7   country     27065 non-null  object
 8   isNeutral   27065 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 1.9+ MB


,date,home_team,away_team,home_score,away_score,tournament,city,country,isNeutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,0
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,0
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,0
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,0
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,0


,date,home_team,away_team,home_score,away_score,tournament,city,country,isNeutral
count,27065,27065,27065,27065.000000,27065.000000,27065,27065,27065,27065.000000
unique,12677,257,263,NaN,NaN,127,1259,255,NaN
top,2003-03-29,Brazil,England,NaN,NaN,Friendly,Kuala Lumpur,Malaysia,NaN
freq,34,443,420,NaN,NaN,11099,629,628,NaN
mean,NaN,NaN,NaN,2.175688,1.606540,NaN,NaN,NaN,0.244375
std,NaN,NaN,NaN,2.224717,2.012111,NaN,NaN,NaN,0.429724
min,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,0.000000
25%,NaN,NaN,NaN,1.000000,0.000000,NaN,NaN,NaN,0.000000
50%,NaN,NaN,NaN,2.000000,1.000000,NaN,NaN,NaN,0.000000
75%,NaN,NaN,NaN,3.000000,2.000000,NaN,NaN,NaN,0.000000


In [4]:
# 1. Konwersja kolumny date na datetime
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

# Sprawdźmy, czy konwersja się powiodła
print(df['date'].dtype)
print(df['date'].min(), '–', df['date'].max())

# 2. Eksploracja wartości w kolumnie tournament
tourn_counts = df['tournament'].value_counts()
display(tourn_counts.head(10))    # Top 10 najczęstszych turniejów
print(f"Łącznie odnotowaliśmy {len(tourn_counts)} unikalnych turniejów.")

# (opcjonalnie) zobaczmy, ile meczów towarzyskich vs World Cup
friendly_games = tourn_counts.get('Friendly', 0)
worldcup_games = tourn_counts.filter(like='World Cup').sum()
print(f"Mecze towarzyskie: {friendly_games}")
print(f"Mecze World Cup (wszystkie edycje): {worldcup_games}")

datetime64[ns]
1872-11-30 00:00:00 – 2003-12-02 00:00:00


,count
tournament,
Friendly,11099
FIFA World Cup qualification,3710
UEFA Euro qualification,1371
African Cup of Nations qualification,1075
Copa América,651
FIFA World Cup,644
Merdeka Tournament,579
British Home Championship,517
African Cup of Nations,384


Łącznie odnotowaliśmy 127 unikalnych turniejów.
Mecze towarzyskie: 11099
Mecze World Cup (wszystkie edycje): 4354


In [5]:
# Wyfiltrujmy unikalne nazwy zawierające "World Cup" i "Euro"
wc_names = [name for name in df['tournament'].unique() if "World Cup" in name]
eu_names = [name for name in df['tournament'].unique() if "Euro" in name]
print("World Cup:", wc_names)
print("Euro:", eu_names)

World Cup: ['FIFA World Cup', 'FIFA World Cup qualification']
Euro: ['Central European International Cup', 'UEFA Euro qualification', 'UEFA Euro']


In [6]:
# 2.1 Zdefiniujmy listy nazw finałów
final_wc = ["FIFA World Cup"]       # dopisz dokładne nazwy z kroku wyżej, jeśli się różnią
final_euro = ["UEFA Euro"]

# 2.2 Dodaj kolumny
df['is_friendly'] = (df['tournament'] == "Friendly").astype(int)
df['is_major'] = df['tournament'].isin(final_wc + final_euro).astype(int)

# 2.3 Sprawdźmy nowe podziały
print("Mecze towarzyskie:", df['is_friendly'].sum())
print("Mecze finałów World Cup + Euro:", df['is_major'].sum())

Mecze towarzyskie: 11099
Mecze finałów World Cup + Euro: 786


In [7]:
# 3.1. Dodajemy kolumnę year
df['year'] = df['date'].dt.year

# 3.2. Filtrowanie meczów finałów (World Cup + Euro)
majors = df[df['is_major'] == 1].copy()

print("Liczba meczów finałowych po filtrowaniu:", majors.shape[0])
majors['tournament'].value_counts()

Liczba meczów finałowych po filtrowaniu: 786


,count
tournament,
FIFA World Cup,644
UEFA Euro,142
